<a href="https://colab.research.google.com/github/Nadjei74/Collab/blob/main/Agents_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries
!pip  install 'crewai[tools]'
!pip  install youtube-transcript-api
!pip  install yt_dlp


In [ ]:
import os
from crewai import Agent
from crewai import Crew, Process
from crewai_tools import YoutubeChannelSearchTool
from crewai import Task

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")


In [ ]:
youtube_tool = YoutubeChannelSearchTool(youtube_channel_handle='@techwithzoum')

Processing videos:   0%|          | 0/65 [00:00<?, ?it/s]ERROR:root:Failed to fetch transcript for video https://www.youtube.com/watch?v=JwZciSgWCyM
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/embedchain/loaders/youtube_video.py", line 29, in load_data
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)
  File "/usr/local/lib/python3.10/dist-packages/youtube_transcript_api/_api.py", line 137, in get_transcript
    return cls.list_transcripts(video_id, proxies, cookies).find_transcript(languages).fetch(preserve_formatting=preserve_formatting)
  File "/usr/local/lib/python3.10/dist-packages/youtube_transcript_api/_transcripts.py", line 292, in fetch
    return _TranscriptParser(preserve_formatting=preserve_formatting).parse(
  File "/usr/local/lib/python3.10/dist-packages/youtube_transcript_api/_transcripts.py", line 358, in parse
    for xml_element in ElementTree.fromstring(plain_data)
  File "/usr/lib/python3.10/xm

In [ ]:

topic_researcher = Agent(
    role ="Topic Researcher",
    goal ="Search for relevant videos on the {topic} from the provided YouTube channel",
    verbose = True,
    memory = True,
    backstory = "EXpert in finding and analyzing Youtube content especially in AI, Data Science, Machine Learning, and Generative AI topics",
    tools = [youtube_tool],
    allow_delegation = True

)

#linkedin Post Agent
Linkedin_Post_Agent =Agent(
    role ="LinkedIn Post Creator",
    goal = "Create a concise LinkedIn post summary from the transcription provided by the Topic Researcher.",
    verbose = True,
    memory = True,
    backstory ="Expert in crafting engaging LinkedIn posts that summarize complex topics and include trending hashtags for maximum visibility.",
    allow_delegation = False
)

# Twitter Agent
twitter_agent = Agent(
    role='Twitter Content Creator',
    goal='Create a short tweet from the transcription provided by the Topic Researcher that capture key points and insights',
    verbose=True,
    memory=True,
    backstory="Specializes in distilling complex information into concise, impactful tweets that resonate with a tech-savvy audience.",
    allow_delegation=False
)

# Blog Writer Agent
blog_writer = Agent(
    role='Blog Writer',
    goal='Write a comprehensive blog post from the transcription provided by the Topic Researcher, covering all necessary sections',
    verbose=True,
    memory=True,
    backstory="Experienced in creating in-depth, well-structured blog posts that explain technical concepts clearly and engage readers from introduction to conclusion.",
    allow_delegation=False
)


**Defining Tasks**

In [ ]:
#Tasks
research_task = Task(
      description = "Identify and analyze videos on the topic {topic} from the specified YouTube channel.",
      expected_output="A complete word by word report on the most relevant video found on the topic {topic}.",
      agent = topic_researcher,
      tools = [youtube_tool]

 )

blog_writing_task= Task(
    description = """ Write a comprehensive blog post based on the transcription provided by the Topic Researcher.
                    The article must include an introduction , step-by-step guides, and conclusion.
                    The overall content must be about 1200 words long.""",
    expected_output ="A markdown-formatted of the blog",
    agent = blog_writer,
    output_file = "blog_post.md"
)

linkedin_post_task = Task(
    description="Create a LinkedIn post summarizing the key points from the transcription provided by the Topic Researcher, including relevant hashtags.",
    expected_output="A markdown-formatted of the LinkedIn post",
    agent=Linkedin_Post_Agent,
    output_file='linkedin-post.md'
)

twitter_task = Task(
    description="Create a tweet from the transcription provided by the Topic Researcher, including relevant hastags.",
    expected_output="A markdown-formatted of the Twitter post",
    agent=twitter_agent,
    output_file='tweets.md'
)


**Running The Crew**

In [ ]:
my_crew = Crew(
    agents = [topic_researcher, Linkedin_Post_Agent, twitter_agent, blog_writer],
    tasks = [research_task, linkedin_post_task, twitter_task, blog_writing_task],
    verbose = True,
    memory = True,
    process = Process.sequential,
    cache = True,
    max_rpm = 100,
    share_crew = True
 )

In [ ]:
topic_of_interest = "'GPT3.5 Turbo Fine-tuning and Graphical Interface"
result = my_crew.kickoff(inputs={'topic':topic_of_interest})
print("-----------------------------")
print(result)


 [2024-08-16 03:16:34][DEBUG]: == Working Agent: Topic Researcher
 [2024-08-16 03:16:34][INFO]: == Starting Task: Identify and analyze videos on the topic 'GPT3.5 Turbo Fine-tuning and Graphical Interface from the specified YouTube channel.


> Entering new CrewAgentExecutor chain...


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.66it/s]


I need to search for relevant videos on the topic "GPT3.5 Turbo Fine-tuning and Graphical Interface" from the YouTube channel @techwithzoum. The best approach is to use the provided tool to perform a search query on this channel.

Action: Search a Youtube Channels content
Action Input: {"search_query": "GPT3.5 Turbo Fine-tuning and Graphical Interface"} 

Relevant Content:
GPT 3.5 turbo model was made available for fine tuning a few days ago and in this tutorial I'll explain how you can create a personal assistant using your custom data the data set I will be using is the Yahoo question answering data sets it has two main columns the question column and the answer column let's get back to the notebook and the first thing to do is to install the data sets package which is used to download data set from hugging face as you can see here the data set is available on hugging phase after this package is installed we import from data set we'll load data set function we provide the name of a d